In [1]:
import time
import json
import pickle
import numpy as np
import pandas as pd
from classes import *
from datetime import datetime
import plotly.graph_objects as go

In [2]:
def get_df(matrix: str, day: int):
    """
    matrix: "air" or "sample"
    day: "day_1", "day_3", ... 
    """
    with open(f"raw_data/day_{day}/{matrix}.bmerawdata") as f:
        j = json.load(f)

    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)
    return df

In [3]:
def get_interpolated_net_data(day: int, start: int, end: int):
    with open(f"data/day_{day}/interpolation_functions.pkl", "rb") as f:
        interp_funcs = pickle.load(f)

    t = np.linspace(start, end, 100)
    net_data = {"t": t}
    for sensor in range(8):
        hps = []
        for hp in range(10):
            y_sample = np.log(interp_funcs["sample"][sensor][hp](t))
            y_air = np.log(interp_funcs["air"][sensor][hp](t))
            y_net = y_air - y_sample
            hps.append(y_net)
        net_data[sensor] = np.array(hps)
    return net_data

In [4]:
def get_tspo_val(date_str, df):
    """ 
    get_tspo_val("2025-07-12 16:26:00", df) -> 2293358
    """
    ts = time.mktime(datetime.strptime(
        date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    ts = int(ts)
    values = df[df["Real time clock"] == ts]["Time Since PowerOn"].values
    return values[0]

In [ ]:
label_datetimes = [
    {
        "day": 0,
        "sample": {
            "start": "2025-07-11 15:55:00",
            "end": "2025-07-11 16:15:00"
        },
        "air": {
            "start": "2025-07-11 15:55:00",
            "end": "2025-07-11 16:15:00"
        }
    },
    {
        "day": 1,
        "sample": {
            "start": "2025-07-12 16:26:00",
            "end": "2025-07-12 16:46:00"
        },
        "air": {
            "start": "2025-07-12 16:26:00",
            "end": "2025-07-12 16:46:00"
        }
    },
    {
        "day": 2,
        "sample": {
            "start": "2025-07-13 16:26:00",
            "end": "2025-07-13 16:46:00"
        },
        "air": {
            "start": "2025-07-13 16:26:00",
            "end": "2025-07-13 16:46:00"
        }
    },
    # {
    #     "day": 3,
    #     "sample": {
    #         "start": "2025-07-14 14:58:00",
    #         "end": "2025-07-14 15:18:00"
    #     },
    #     "air": {
    #         "start": "2025-07-14 14:58:00",
    #         "end": "2025-07-14 15:18:00"
    #     }
    # },
]

In [ ]:
days = {}
for label_datetime in label_datetimes:
    day_idx = label_datetime["day"]

    sample_df = get_df("sample", day_idx)
    sample_start_tstamp = get_tspo_val(label_datetime["sample"]["start"], sample_df)
    sample_end_tstamp = get_tspo_val(label_datetime["sample"]["end"], sample_df)

    days[day_idx] = get_interpolated_net_data(0, sample_start_tstamp, sample_end_tstamp)

In [21]:
print(days[0]["t"][0], days[0]["t"][-1])
print(days[1]["t"][0], days[1]["t"][-1])
print(days[2]["t"][0], days[2]["t"][-1])

2235529.0 3435298.0
2345489.0 3545380.0
2303206.0 3503321.0


In [18]:
def plot_days_net(sensor_data, sensor_idx):
    title = f"Net Sensor {sensor_idx} Readings"
    fig = go.Figure()

    for day_idx, day_data in sensor_data.items():
        t = day_data["t"]
        sensor_data = day_data[sensor_idx]
        for y in sensor_data:
            fig.add_trace(go.Scatter(x=t,
                                     y=y,
                                     mode="lines",
                                     showlegend=False))

        fig.add_vrect(x0=t[0],
                      x1=t[-1],
                      annotation_text=f"day {day_idx}",
                      annotation_position="top left",
                      line_width=0.5)

    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=600, width=900,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.update_yaxes(type="log")
    fig.show()

In [20]:
plot_days_net(days, 1)